In [1]:
library(magrittr)
library(tidyverse)

results <- '../results/'

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract()   masks magrittr::extract()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::lag()       masks stats::lag()
✖ purrr::set_names() masks magrittr::set_names()


In [2]:
dmgwas <- read_tsv(paste0(results,'dmgwas/selected_genes.dmgwas.txt'), col_types = 'c')$gene
sigmod <- read_tsv(paste0(results,'sigmod/selected_genes.sigmod.txt'), col_types = 'c')$gene
heinz <- read_tsv(paste0(results,'heinz/selected_genes.heinz.txt'), col_types = 'c')$gene
hotnet2 <- read_tsv(paste0(results,'hotnet2/selected_genes.hotnet2.tsv'), col_types = 'cc')$gene
lean <- read_tsv(paste0(results,'lean/scored_genes.lean.txt'), col_types = 'cdddddddd') %>%
    filter(PLEAN < 0.05) %>%
    .$Gene
consensus <- read_tsv(paste0(results,'consensus/consensus.tsv'), col_types = 'clllllii')$name

Warning message:
“Unnamed `col_types` should have the same length as `col_names`. Using smaller of the two.”Warning message:
“556 parsing failures.
row         col           expected            actual                                 file
  1 betweenness 1/0/T/F/TRUE/FALSE 10665.95418836259 '../results/consensus/consensus.tsv'
  1 degree      1/0/T/F/TRUE/FALSE 36                '../results/consensus/consensus.tsv'
  1 p           1/0/T/F/TRUE/FALSE 0.014529854701453 '../results/consensus/consensus.tsv'
  1 hotnet2     an integer         TRUE              '../results/consensus/consensus.tsv'
  1 heinz       an integer         FALSE             '../results/consensus/consensus.tsv'
... ........... .................. ................. ....................................
See problems(...) for more details.
”Warning message:
“Unknown or uninitialised column: 'name'.”

In [3]:
scones_gs <- read_tsv(paste0(results,'scones/cones_gs.tsv'), col_types = "ciiiccdli") %>%
    filter(selected) %>%
    .$snp
scones_gm <- read_tsv(paste0(results,'scones/cones_gm.tsv'), col_types = "ciiiccdli") %>%
    filter(selected) %>%
    .$snp
scones_gi <- read_tsv(paste0(results,'scones/cones_gi.tsv'), col_types = "ciiiccdli") %>%
    filter(selected) %>%
    .$snp

# Supplementary file 1: selected genes

In [4]:
snp2gene <- read_tsv(paste0(results,'preprocessing/snp2hgnc.tsv'), col_types = 'cc')
scones_gs_gene <- filter(snp2gene, snp %in% scones_gs) %>% .$gene
scones_gm_gene <- filter(snp2gene, snp %in% scones_gm) %>% .$gene
scones_gi_gene <- filter(snp2gene, snp %in% scones_gi) %>% .$gene

In [5]:
read_tsv(paste0(results,'preprocessing/scored_genes.vegas.txt'), 
         col_types = 'iciddddddcd') %>%
    mutate(Consensus = ifelse(Gene %in% consensus, 'Yes', 'No'),
           dmGWAS = ifelse(Gene %in% dmgwas, 'Yes', 'No'),
           heinz = ifelse(Gene %in% heinz, 'Yes', 'No'),
           HotNet2 = ifelse(Gene %in% hotnet2, 'Yes', 'No'),
           LEAN = ifelse(Gene %in% lean, 'Yes', 'No'),
           SConES_GS = ifelse(Gene %in% scones_gs_gene, 'Yes', 'No'),
           SConES_GM = ifelse(Gene %in% scones_gm_gene, 'Yes', 'No'),
           SConES_GI = ifelse(Gene %in% scones_gi_gene, 'Yes', 'No'),
           SigMod = ifelse(Gene %in% sigmod, 'Yes', 'No')) %>%
    write_tsv('files/sfile_1.tsv')

# Supplementary file 2: selected SNPs

In [6]:
snp_assoc <- read_tsv(paste0(results, 'conventional_gwas/univariate_models.no_covars.tsv'), 
         col_types = 'icdccccddd') %>%
    rename(Chr = CHR) %>%
    select(-A1, -F_A, -F_U, -A2)

In [7]:
dmgwas_snp <- filter(snp2gene, gene %in% dmgwas) %>% .$snp
sigmod_snp <- filter(snp2gene, gene %in% sigmod) %>% .$snp
heinz_snp <- filter(snp2gene, gene %in% heinz) %>% .$snp
hotnet2_snp <- filter(snp2gene, gene %in% hotnet2) %>% .$snp
lean_snp <- filter(snp2gene, gene %in% lean) %>% .$snp
consensus_snp <- filter(snp2gene, gene %in% consensus) %>% .$snp

In [8]:
snp_assoc %>%
    mutate(consensus = ifelse(SNP %in% consensus_snp, 'Yes', 'No'),
           dmGWAS = ifelse(SNP %in% dmgwas_snp, 'Yes', 'No'),
           heinz = ifelse(SNP %in% heinz_snp, 'Yes', 'No'),
           HotNet2 = ifelse(SNP %in% hotnet2_snp, 'Yes', 'No'),
           LEAN = ifelse(SNP %in% lean_snp, 'Yes', 'No'),
           SConES_GS = ifelse(SNP %in% scones_gs, 'Yes', 'No'),
           SConES_GM = ifelse(SNP %in% scones_gm, 'Yes', 'No'),
           SConES_GI = ifelse(SNP %in% scones_gi, 'Yes', 'No'),
           SigMod = ifelse(SNP %in% sigmod_snp, 'Yes', 'No')) %>%
    write_tsv('files/sfile_2.tsv')